In [2]:
import importlib
import utilities

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mplcursors
import pandas as pd
from matplotlib.lines import Line2D
from scipy.stats import linregress 
from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq
import scipy.signal as sig
from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq
import sys
import pickle
import os


In [3]:
#REMARK: WHEN RELOADING THIS FILE IN THE OTHER NOTEBOOK,THE ISINSTANCE WONT RECOGNIZE THE CLASSES
#BECAUSE THE VERSION DEFINED WAS THE VERSION FROM THE PREVIOUS VERSION OF THE NOTEBOOK !!!!!!!

# free parameters:
# order of windowing
# chi2 to decide parts of lc to take for averaging

#TODOS:
#NEED MAXMINEFFICIENT TABLE FOR AVERAGE SHAPLET AND WITH NORMALIZED COMPARATOR FOR RHO AND ALL
#TEST THE CHI2 < 2 FOR CLEANING

In [4]:
#define the Data class: 

In [5]:
importlib.reload(utilities)

class Data:
    def __init__(self,lcnumber):
        self.lcnumber=lcnumber
        self.datatable=None
        self.dataclass=None
        self.__load()
        self.cumsumLow=np.cumsum(self.getcolumn("low"))
        self.cumsumMid=np.cumsum(self.getcolumn("mid"))
        self.cumsumHigh=np.cumsum(self.getcolumn("high"))
        self._alpha={"low":None,"mid":None,"high":None}
        self._beta={"low":None,"mid":None,"high":None}

    def _findalpha(self,column):
         self._alpha[column]=utilities.findalpha(self.getcolumn(column))

    def _findbeta(self,column):
        self._beta[column]=utilities.findbeta(self.getcolumn(column))
    
    def alpha(self,column):
        if(self._alpha[column] is None):
            self._findalpha(column)
        return self._alpha[column]

    def beta(self,column):
        if(self._beta[column] is None):
            self._findbeta(column)
        return self._beta[column]
    
    def __load(self):
        self.datatable=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",skiprows=[0, 1], header=None)
        self.datatable.columns=['time', 'total','low','mid','high'] 
        
        firstline=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",nrows=1, header=None).iloc[0,0]
        prefix="# class : "
        self.dataclass=firstline[len(prefix):].strip()

    
    def low(self):
        return np.array(self.datatable["low"])

    def mid(self):
        return np.array(self.datatable["mid"])

    def high(self):
        return np.array(self.datatable["high"])

    def getcolumn(self,column):
        if(column=="low"):
            return self.low()
        if(column=="mid"):
            return self.mid()
        if(column=="high"):
            return self.high()
            
    def cumsum(self,column):
        if(column=="low"):
            return self.cumsumLow
        if(column=="mid"):
            return self.cumsumMid
        if(column=="high"):
            return self.cumsumHigh
        

In [6]:
class normalizedData(Data):
    def __init__(self,lcnumber):
        self.lcnumber=lcnumber
        self.datatable=None
        self.dataclass=None
        self.__normalizedload()
        self.cumsumLow=np.cumsum(self.getcolumn("low"))
        self.cumsumMid=np.cumsum(self.getcolumn("mid"))
        self.cumsumHigh=np.cumsum(self.getcolumn("high"))
        self._alpha={"low":None,"mid":None,"high":None}
        self._beta={"low":None,"mid":None,"high":None}
        
    def __normalizedload(self):
        self.datatable=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",skiprows=[0, 1], header=None)
        self.datatable.columns=['time', 'total','low','mid','high'] 
        for c in self.datatable.columns:
            self.datatable[c]=utilities.normalize(self.datatable[c])
            
        firstline=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",nrows=1, header=None).iloc[0,0]
        prefix="# class : "
        self.dataclass=firstline[len(prefix):].strip()

In [7]:
#define Shaplet class

In [8]:
importlib.reload(utilities)
class Shaplet:
    def __init__(self,lcnumber,column,start,length,alpha=-1,mean=-1,isaclean=False,series=None,isnormalized=False):
        self.isnormalized=isnormalized
        self.isaclean=isaclean
        if not (series is None):
            self.series=series
            self.lcnumber=lcnumber              #THIS SHOULD CHANGE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            self.column=column
            self.start=start
            self.length=len(self.series)
        else:
            self.lcnumber=lcnumber
            self.start=start
            self.column=column
            self.length=length
            self.end=start+length
            self.series=None
            if(isnormalized):
                self.load(normalizedData(self.lcnumber))
            else:
               # print("get here too")
                self.load(Data(self.lcnumber))
        self.sum=np.sum(self.series)
        self.alpha=utilities.findalpha(self.series)
        self.beta=utilities.findbeta(self.series)
        self.mean=mean
        self.alpha=alpha
        
        
    def normalizeserie(self):
        self.series=self.alpha*self.series+self.beta

    def getnormalizedserie(self):
        return self.alpha*self.series+self.beta
    
    def __len__(self):
        return self.length

    #i will leave it right now public,we will see what we can do in the future
    def load(self,data):
        if data.lcnumber != self.lcnumber:
            raise ValueError("Error: lcnumber values do not match.")  # Raising an error properly
        if(self.series is None):
            #print(data.getcolumn(self.column)[self.start:self.end])
            self.series=data.getcolumn(self.column)[self.start:self.end]
            #print(self.series)

In [9]:
importlib.reload(utilities)
class Comparator:
    def __init__(self,data:Data,column,shaplet:Shaplet):
        self.data=data
        self.shaplet=shaplet
        self.column=column
        self.signallength=len(data.getcolumn(column))
        
        
        self._chi2=None
        self._minimaspositions=None
        self._minimas=None
        
        self._maxCut=None
        self._minCut=None
        self._efficientCut=None
        self.cumsum=self.data.cumsum(self.column)
        self.alpha=np.ones( max(len(self.cumsum) - len(self.shaplet)+1,0) )
        self._scaledchi2=None
    
    def __computealpha(self):
        s=self.cumsum
        signal=self.data.getcolumn(self.column)
        N=len(self.shaplet)
        alpha=np.array([s[N-1]])
        alpha=np.concatenate((alpha,self.cumsum[N:]-self.cumsum[:-N]))
        alpha=alpha/np.sum(self.shaplet.series) 
        return alpha

    def scaledchi2measure(self,signal,shapletseries):
        #NEED TO CHECK IF THE SHAPLET SERIES IS LOADED IF NOT THEN I LL HAE AN ERROR 
        if(len(signal)<len(shapletseries)): #By convention the difference is -1 if the sizes arent matching
            return np.array(-1)
            
        difference=[] #diffrence between shaplet & signal
        mean_=np.mean(shapletseries)
        shapletlength=len(shapletseries)
        for i in range(0,len(signal)-len(shapletseries)):
            alpha=self.alpha[i]
            diff=np.sum(np.power(signal[i:i+len(shapletseries)]-alpha*shapletseries,2) / ((alpha+alpha**2)*mean_*8*len(shapletseries)))
            difference.append(diff)
        difference=np.array(difference)
        return difference
        
    @staticmethod
    def chi2measure(signal,shaplet:Shaplet,alpha=None,mean=None):
    #NEED TO CHECK IF THE SHAPLET SERIES IS LOADED IF NOT THEN I LL HAE AN ERROR !!!------------------------------------------------------------------------
        if(len(signal)<len(shaplet)): #By convention the difference is -1 if the sizes arent matching
            return np.array(-1)
        difference=[] #diffrence between shaplet & signal
        mean_=np.mean(shaplet.series)
        shapletlength=len(shaplet)
        for i in range(0,len(signal)-len(shaplet)):
            if(not shaplet.isnormalized):
                diff=np.sum(np.power(signal[i:i+len(shaplet)]-shaplet.series,2) / (2*mean_*8*len(shaplet)) )
            else:
                diff=np.sum(np.power(signal[i:i+len(shaplet)]-shaplet.series,2) / (2*(shaplet.mean)*(shaplet.alpha**2)*8*len(shaplet)) )
            difference.append(diff)
        difference=np.array(difference)
        return difference

    def __chi2measure(self):
        if self._chi2 is None:
            self._chi2=Comparator.chi2measure(self.data.getcolumn(self.column),self.shaplet)

    def __scaledchi2measuring(self):
        if self._scaledchi2 is None:
            self._scaledchi2=self.scaledchi2measure(self.data.getcolumn(self.column),self.shaplet.series)
    
    def chi2(self):
        self.__chi2measure()
        return self._chi2

    def scaledchi2(self):
        self.__scaledchi2measuring()
        return self._scaledchi2
    
    def __findminimas(self):
        self.__chi2measure()
        
        if np.array_equal(self._chi2,np.array(-1)):
            self._minimas=np.array(-1)
            self._minimaspositions=np.array(-1)
            self._minCut=-1
            self._maxCut=-1
            
        elif ( self._minimaspositions is None ):
            #sig.argrelmin(Comparator.chi2measure(signal,shaplet),order=int(len(shaplet)/2),mode="wrap")[0]
            if(len(self.chi2())<int(len(self.shaplet)/2)):
                self._minimaspositions=np.array([np.argmin(self.chi2())])
                self._minimas=self._chi2[self._minimaspositions]
                self._minCut=min(self._chi2)
                self._maxCut=max(self._chi2)
            else:
                self._minimaspositions=sig.argrelmin(self.chi2(),order=int(len(self.shaplet)/2),mode="wrap")[0]
                self._minimas=self._chi2[self._minimaspositions]
                self._minCut=min(self._minimas)
                self._maxCut=max(self._minimas)
    
    def minimaspositions(self,cut=None):
        self.__findminimas()
        return self._minimaspositions if cut is None else self._minimaspositions[self._minimas<=cut]
    
    def minimas(self,cut=None):
        self.__findminimas()
        return self._minimas if cut is None else self._minimas[self._minimas<=cut]

    def findcut(self,precision=1e-2):
        if not (self._efficientCut is None):
            return

        if(np.array_equal(self._chi2,np.array(-1))):
            self._efficientCut=-1
            return

        if not utilities.noholes(self.__positionsforcut(self._maxCut),len(self.shaplet)):  
            self._efficientCut=-1
            return
        
        self.__findminimas() 
        low=self._minCut
        high=self._maxCut
   #     print(low)
  #      print(high) 
 #       print(precision)
        while high - low > precision:
  #          print(low,end=" it got inside ")
 #           print(high)
            mid = (low + high) / 2
            if utilities.noholes(self.__positionsforcut(mid),len(self.shaplet)):  
                high = mid  # Move left if `foo(mid)` is True
            else:
                low = mid   # Move right if `foo(mid)` is False
                
        self._efficientCut=high

    def __positionsforcut(self,cut):
        return sorted(self._minimaspositions[self._minimas<=cut])

    def min(self):
        self.__findminimas()
        return self._minCut

    def efficient(self):
        self.__findminimas()
        self.findcut()
        return self._efficientCut
    
    def max(self):
        self.__findminimas()
        return self._maxCut

    def coveredarea(self):
        area=0
        minpositions=self.minimaspositions()
        for i in range(len(minpositions)):
            if(i==len(minpositions)-1):
                if(self.signallength-minpositions[i]>=len(self.shaplet)):
                    area+=len(self.shaplet)
                else:
                    area+=self.signallength-minpositions[i]
            else:
                el=minpositions[i]
                nextel=minpositions[i+1]
                if( el+len(self.shaplet) > nextel ):
                    area+=(nextel-el)
                else:
                    area+=len(self.shaplet)
       #     print(area)      
        return area

In [10]:
class scaledComparator(Comparator):
    def __init__(self,data:Data,column,shaplet:Shaplet):
        super().__init__(data,column,shaplet)
        self.alpha=super()._Comparator__computealpha()
        self._chi2=self.scaledchi2()

In [11]:
class normalizedComparator(Comparator):
    def __init__(self,data:Data,column,shaplet:Shaplet):
        # definition of a,b,c and d:
        # a*S+b=c*shaplet+d
        super().__init__(data,column,shaplet)
        self.a=self.data.alpha(self.column) 
        self.b=self.data.beta(self.column) 
        self.c=self.shaplet.alpha
        self.d=self.shaplet.beta

        self._chi2= self.normalizedchi2(self.data.getcolumn(self.column),self.shaplet.series)
    
    def normalizedchi2(self,signal,shapletseries):
        #NEED TO CHECK IF THE SHAPLET SERIES IS LOADED IF NOT THEN I LL HAE AN ERROR 
        if(len(signal)<len(shapletseries)): #By convention the difference is -1 if the sizes arent matching
            return np.array(-1)
            
        difference=[] #diffrence between shaplet & signal
        mean_=np.mean(shapletseries)
        shapletlength=len(shapletseries)
        for i in range(0,len(signal)-len(shapletseries)):
            alpha=self.alpha[i]
            #YOU ARE HERE
            #MAKE SURE YOU ARE PROGRAMMING THE RIGHT DIFFERENCE FORMULAE !!
            diff=np.sum(np.power(self.a*signal[i:i+len(shapletseries)]+self.b-self.c*shapletseries-self.d,2) /
                        (( (self.a**2)*np.mean(signal[i:i+len(shapletseries)]) + (self.c **2)*mean_ )*8*len(shapletseries)))
            difference.append(diff)
        difference=np.array(difference)
        return difference

In [22]:
class Visualizer:
    def __init__(self):
        pass


    def superpose(self,comparator:Comparator,xlim=None,title=None,cut=None,cleaned=False,cleanedopacity=0.8,showing=True,
                  initiate=True,draw=True,color='r'):
        #ADD A TITLE FOR THIS PLOT
        if cleaned==True:
            opacity=cleanedopacity
        else:
            opacity=0.5
        shaplet=comparator.shaplet
        data=comparator.data
        if(initiate):
            plt.figure()   
        signal=data.getcolumn(comparator.column)
        if(xlim is not None):
            plt.xlim(np.array(xlim)/8)
        if(title is not None):
            plt.title(title)
        else:
            plt.title("lc number: "+str(comparator.data.lcnumber))
        minimaspositions=comparator.minimaspositions(cut)
        
        #just to test:
        print(minimaspositions)
        #
     #   minimaspositions=sig.argrelmin(Comparator.chi2measure(signal,shaplet),order=int(len(shaplet)/2),mode="wrap")[0]
        if(draw):
            plt.plot(np.array(range(len(signal)))/8,signal,color='b')  #EVERYTHING IN SECONDS FROM NOW ON !!!
  #                plt.plot(np.array( range(index,index+min(len(shaplet),comparator.signallength)) )/8,
 #                             shaplet.series[:min(len(shaplet),comparator.signallength)],color='r',alpha=0.5)
#


        # 
        min_chi2_index = min(minimaspositions, key=lambda idx: comparator.chi2()[idx])
        
        if (isinstance(comparator,normalizedComparator)):
             for i in range(len(minimaspositions)):
                index=minimaspositions[i]
                if(i==len(minimaspositions)-1 and len(shaplet)+index >=comparator.signallength):
                    plt.plot(np.array(range(index,comparator.signallength))/8,
    abs((comparator.c/comparator.a))*shaplet.series[:comparator.signallength-index]+(comparator.d-comparator.b)/comparator.a,
                    color=color,alpha=opacity)
                else:
                    plt.plot(np.array(range(index,index+len(shaplet)))/8,comparator.c*shaplet.series+comparator.d,
                             color=color,alpha=opacity)
                    
        else:
            for i in range(len(minimaspositions)):
                index=minimaspositions[i]
                #
                # Opacity and color logic
                current_opacity = 1.0 if index == min_chi2_index else opacity
                current_color =  color  #'green' if index == min_chi2_index else

                if i == len(minimaspositions) - 1 and len(shaplet) + index >= comparator.signallength:
                    plt.plot(np.array(range(index, comparator.signallength)) / 8,
                             comparator.alpha[index] * shaplet.series[:comparator.signallength - index],
                             color=current_color, alpha=current_opacity)
                else:
                    plt.plot(np.array(range(index, index + len(shaplet))) / 8,
                             comparator.alpha[index] * shaplet.series,
                             color=current_color, alpha=current_opacity)
                    
        plt.xlabel("time in s")
        plt.ylabel("intensity count /s")
        if(showing):
            plt.show()

    def plotshapletsonlc(lightcurve, orderedshapletlist, indexes, shapletdictionary, color_dict,min_chi2_index=None, opacity=0.3):
        # Step 1: Create a unique color for each shaplet number
        unique_shaplets = list(set(orderedshapletlist))
        n_shaplets = len(unique_shaplets)

        # Step 2: Plot the lightcurve
        plt.plot(np.arange(len(lightcurve)) / 8, lightcurve, color='blue', alpha=0.7, label='Lightcurve')

        # Step 3: Plot each shaplet at the given start index
        for i, shaplet_num in enumerate(orderedshapletlist):
            start_idx = indexes[i]
            shaplet = shapletdictionary[shaplet_num]  # Assuming shapletdictionary[shaplet_num] has attribute 'series' and 'length' or just is a 1D array

            # Current color from dictionary
            current_color = color_dict[shaplet_num]

            # Current opacity (full opacity if this is the min chi2 index, else reduced)
            current_opacity = 1.0 if (min_chi2_index is not None and i == min_chi2_index) else opacity

            # Determine end index
            end_idx = start_idx + len(shaplet)
            if end_idx > len(lightcurve):
                # Trim shaplet if it runs past the lightcurve length
                plot_x = np.arange(start_idx, len(lightcurve)) / 8
                plot_y = shaplet[:len(lightcurve) - start_idx]
            else:
                plot_x = np.arange(start_idx, end_idx) / 8
                plot_y = shaplet

            # Plot the shaplet on the lightcurve
            plt.plot(plot_x, plot_y, color=current_color, alpha=current_opacity)

        plt.legend()
        plt.xlabel('Time')
        plt.ylabel('Signal')
        plt.title('Lightcurve with Shaplets Overlay')
        plt.show()
                    
                    
                    
    def superposemany(self,comparators,xlim=None,title=None,cut=None,cleaned=None):
        plt.figure() #comparator have the same lc
        signal=comparators[0].data.getcolumn(comparators[0].column)
        plt.plot(np.array(range(len(signal)))/8,signal,color='b')  #EVERYTHING IN SECONDS FROM NOW ON !!!
        n = len(comparators)
        color_map = plt.cm.get_cmap('tab20', n) 
        i=0
        for i,comparator in enumerate(comparators):  # DONT TO SAVE NOW THIS MESSAGE IS NEW !!------------------------------
            color = color_map(i)
            self.superpose(comparator,cleaned=False,showing=False,initiate=False,draw=False,color=color) #draw for srawing the signal
        plt.show()
        
    def superposeatminima(self,comparator:Comparator,xlim=None,title=None):
        self.superpose(comparator,cut=comparator.min())
    
    def showchi2(self,comparator:Comparator):
        plt.figure()
        plt.title("lc number: "+str(comparator.data.lcnumber))
        plt.plot(np.array(range(len(comparator.chi2())))/8,comparator.chi2())
        plt.xlabel("window index")
        plt.ylabel("chi^2")
        plt.show()

    def shownormalized(self,arr,title=" "):
        if(isinstance(arr,Shaplet)):
            utilities.show(utilities.normalize(arr.series),title=title)
        else:
            utilities.show(utilities.normalize(arr),title=title)
    
        
    def showscaledchi2(self,comparator:Comparator):
        plt.figure()
        plt.title("lc number: "+str(comparator.data.lcnumber))
        plt.plot(np.array(range(len(comparator.scaledchi2())))/8,comparator.scaledchi2())
        plt.xlabel("window index")
        plt.ylabel("scaled chi^2")
        plt.show()

    
    def showlow(self,data:Data):
        plt.figure()
        plt.title("lc number: "+str(data.lcnumber))
        plt.plot(data.low())
        plt.xlabel("time in s")
        plt.ylabel("intensity count /s")
        plt.show()

    def shownormalizedlow(self,data:Data):
        plt.figure()
        plt.title("lc number: "+str(data.lcnumber))
        plt.plot(data.alpha("low")*data.low()+data.beta("low"))
        plt.xlabel("time in s")
        plt.ylabel("intensity count /s")
        plt.show()
    
    def show(self,series,title=" "):
        if(isinstance(series,Shaplet)):
            utilities.show(series.series,title=title)
        else:
            utilities.show(series,title=title)

In [13]:
def minmaxefficient(list_,comparators:list[Comparator]):
    "returns dataframe of min max and efficient cuts for each comparator in comparators"
    cuts=pd.DataFrame(columns=['index','min', 'efficient', 'max'])
  #  print(cuts)
    for index in list_:
 #       print(index)
        cuts.loc[len(cuts)]=[index,comparators[index].min(),comparators[index].efficient(),comparators[index].max()]
    return cuts

In [14]:
def comparewith(list_,column,shaplet:Shaplet):
    "returns a list of comparators of Data(i) and shaplet where i reperents elements in list_"
    comparators=[]
    for i in list_:
        comparators.append(Comparator(Data(i),column,shaplet))
    return comparators

def comparewithnormalized(list_,column,shaplet:Shaplet):
    "returns a list of comparators of Data(i) and shaplet where i reperents elements in list_"
    comparators=[]
    for i in list_:
        comparators.append(Comparator(normalizedData(i),column,shaplet))
    return comparators


def scaledcomparewith(list_,column,shaplet:Shaplet):
    "returns a list of comparators of Data(i) and shaplet where i reperents elements in list_"
    comparators=[]
    for i in list_:
        comparators.append(scaledComparator(Data(i),column,shaplet))
    return comparators

def normalizedcomparewith(list_,column,shaplet:Shaplet):
    "returns a list of comparators of Data(i) and shaplet where i reperents elements in list_"
    comparators: List[normalizedComparator] = []
    for i in list_:
        comparators.append(normalizedComparator(Data(i),column,shaplet))
    return comparators
    

#  With what lcs should i compare to find best matches to then take the mean ????

In [15]:
def cleanormalized(shaplet,column,samedir=True,cut=2):
    #TODO: NEED TO HANDLE SHAPLETS DEFINED WITHOUT ANY LCNUMBER !!!
    #cleaning is bad for lambda class,need to handle that !!  Since number of elements isnt that much
    
  #  print("updated") # to eraze later
   # print("len shap",len(shaplet))
    print("updated 5")
    if(not samedir):
        class2key=utilities.loadjson("../class2key")  
        key2class=utilities.loadjson("../key2class")
    else:
        class2key=utilities.loadjson("class2key")  
        key2class=utilities.loadjson("key2class")  
        
 #   print(key2class[str(shaplet.lcnumber)])
    comparators=normalizedcomparewith(class2key[key2class[str(shaplet.lcnumber)]],column,shaplet) # reference for cleaning ???
    print(len(comparators))                                                                                        # I have chosen the shaplet class
    D=[]
    for comparator in comparators:
        try:
            for minpos in comparator.minimaspositions():
                if(comparator.chi2()[minpos]<cut):
                    D.append((comparator.data.lcnumber,minpos))
        except:
            #TODO: I WILL HANDLE ERRORS LATER,MORE LIKELY DUE TO THE LENGTH COMPARISON THING
            continue
    Sum=np.zeros(len(shaplet))
    print(len(D))
    for el in D:
        lcnumber=el[0]
        minpos=el[1]
        
  #      utilities.show(utilities.normalize(Data(lcnumber).getcolumn("low")[minpos:minpos+len(shaplet)]))

        Sum+=utilities.normalize(Data(lcnumber).getcolumn("low")[minpos:minpos+len(shaplet)]) #NORMALIZE The serie first
    res=(Sum + utilities.normalize(shaplet.series))/(1+len(D))
    
    
    
    
    return (Shaplet(shaplet.lcnumber,column,shaplet.start,len(shaplet),series=res),D) #change parameters to None,or depending on the clean shaplet get the original

In [16]:
def clean(shaplet,column,samedir=True,cut=2):
    #TODO: NEED TO HANDLE SHAPLETS DEFINED WITHOUT ANY LCNUMBER !!!
    #cleaning is bad for lambda class,need to handle that !!  Since number of elements isnt that much
    
  #  print("updated") # to eraze later
   # print("len shap",len(shaplet))
    #print("updated 5")
    if(not samedir):
        class2key=utilities.loadjson("../class2key")  
        key2class=utilities.loadjson("../key2class")
    else:
        class2key=utilities.loadjson("class2key")  
        key2class=utilities.loadjson("key2class")  
        
 #   print(key2class[str(shaplet.lcnumber)])
    comparators=comparewith(class2key[key2class[str(shaplet.lcnumber)]],column,shaplet) # reference for cleaning ???
                                                                                            # I have chosen the shaplet class
    D=[]
    for comparator in comparators:
        try:
            for minpos in comparator.minimaspositions():
                if(comparator.chi2()[minpos]<cut):
                    D.append((comparator.data.lcnumber,minpos))
        except:
            #TODO: I WILL HANDLE ERRORS LATER,MORE LIKELY DUE TO THE LENGTH COMPARISON THING
            continue
    Sum=np.zeros(len(shaplet))
    for el in D:
        lcnumber=el[0]
        minpos=el[1]
        
  #      utilities.show(utilities.normalize(Data(lcnumber).getcolumn("low")[minpos:minpos+len(shaplet)]))

        Sum+=utilities.normalize(Data(lcnumber).getcolumn("low")[minpos:minpos+len(shaplet)]) #NORMALIZE The serie first
    res=(Sum + utilities.normalize(shaplet.series))/(1+len(D))
    
    
    
    
    return (Shaplet(shaplet.lcnumber,column,shaplet.start,len(shaplet),series=res),D) #change parameters to None,or depending on the clean shaplet get the original

In [17]:
"""
key2class=utilities.loadjson("key2class")
class2key=utilities.loadjson("class2key")
visualizer=Visualizer()"""

'\nkey2class=utilities.loadjson("key2class")\nclass2key=utilities.loadjson("class2key")\nvisualizer=Visualizer()'

In [18]:
"""
for i in class2key["rho"]:
    alpha[i]=utilities.findalpha(Data(i).getcolumn("low"))
    normalizeddata[i]=normalizedData(i)
    """

'\nfor i in class2key["rho"]:\n    alpha[i]=utilities.findalpha(Data(i).getcolumn("low"))\n    normalizeddata[i]=normalizedData(i)\n    '

In [19]:
#alpha=utilities.findalpha(Data(432).getcolumn("low"))
#mean=np.mean(Data(432).getcolumn("low")[0:300])
#shaplet0=Shaplet(432,"low",0,300,mean=mean,alpha=alpha,isnormalized=True) #column,start,length,isaclean=False,series=None):

In [20]:
#comp=normalizedComparator(normalizeddata[181],"low",shaplet0)

In [71]:
"""
%matplotlib widget
visualizer.superpose(comp)
"""

'\n%matplotlib widget\nvisualizer.superpose(comp)\n'

# computed covered area of the shaplet on rho (change the function to count for shaplet that dont end fully) 

# PROGRAM NOW THE DIFFERENT SHAPLETS ESPECIALLY THE SMALL ONES

In [72]:
# find if data is continuous

In [73]:
"""
datatable=pd.read_csv("classified_lcs/grs1915_lc"+str(lcnumber)+".txt",sep="	",skiprows=[0, 1], header=None)
datatable.columns=['time', 'total','low','mid','high'] 

firstline=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",nrows=1, header=None).iloc[0,0]
prefix="# class : "
self.dataclass=firstline[len(prefix):].strip()
"""

'\ndatatable=pd.read_csv("classified_lcs/grs1915_lc"+str(lcnumber)+".txt",sep="\t",skiprows=[0, 1], header=None)\ndatatable.columns=[\'time\', \'total\',\'low\',\'mid\',\'high\'] \n\nfirstline=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="\t",nrows=1, header=None).iloc[0,0]\nprefix="# class : "\nself.dataclass=firstline[len(prefix):].strip()\n'

In [75]:
"""
import pandas as pd
import os

folder = "classified_lcs"
num_files = len(class2key["rho"])  # Change this to your actual number of light curves

last_times = []
time_issues = []

for lcnumber in range(num_files):
    filepath = os.path.join(folder, "grs1915_lc"+str(class2key["rho"][lcnumber])+".txt")
    
    if not os.path.isfile(filepath):
        print(f"❌ File not found: {filepath}")
        continue

    datatable = pd.read_csv(filepath, sep="\t", skiprows=[0, 1], header=None)
    datatable.columns = ['time', 'total', 'low', 'mid', 'high']
    
    time_col = datatable['time']
    
    # Check if time is increasing
    if not time_col.is_monotonic_increasing:
        time_issues.append((filepath, "Time column not strictly increasing"))
    
    # Check if time step is constant
    time_diffs = time_col.diff().dropna()
    if time_diffs.nunique() > 1:
        time_issues.append((filepath, "Time step not constant"))
    
    first_time = time_col.iloc[0]
    last_time = time_col.iloc[-1]
    last_times.append((filepath, first_time, last_time))

# Check if last_time < next_first_time
cross_file_issues = []
for i in range(len(last_times) - 1):
    current_last = last_times[i][2]
    next_first = last_times[i + 1][1]
    if current_last >= next_first:
        cross_file_issues.append((
            last_times[i][0],
            last_times[i + 1][0],
            f"last_time={current_last} >= next_first_time={next_first}"
        ))

# Summary
if not time_issues and not cross_file_issues:
    print("✅ All time columns are continuous and sequential across files.")
else:
    if time_issues:
        print("\n⚠️ Time continuity issues:")
        for issue in time_issues:
            print(issue)
    if cross_file_issues:
        print("\n⚠️ Cross-file time ordering issues:")
        for issue in cross_file_issues:
            print(issue)
"""

'\nimport pandas as pd\nimport os\n\nfolder = "classified_lcs"\nnum_files = len(class2key["rho"])  # Change this to your actual number of light curves\n\nlast_times = []\ntime_issues = []\n\nfor lcnumber in range(num_files):\n    filepath = os.path.join(folder, "grs1915_lc"+str(class2key["rho"][lcnumber])+".txt")\n    \n    if not os.path.isfile(filepath):\n        print(f"❌ File not found: {filepath}")\n        continue\n\n    datatable = pd.read_csv(filepath, sep="\t", skiprows=[0, 1], header=None)\n    datatable.columns = [\'time\', \'total\', \'low\', \'mid\', \'high\']\n    \n    time_col = datatable[\'time\']\n    \n    # Check if time is increasing\n    if not time_col.is_monotonic_increasing:\n        time_issues.append((filepath, "Time column not strictly increasing"))\n    \n    # Check if time step is constant\n    time_diffs = time_col.diff().dropna()\n    if time_diffs.nunique() > 1:\n        time_issues.append((filepath, "Time step not constant"))\n    \n    first_time = 